In [1]:
import os
import requests
import pandas as pd
import urllib.request
from tqdm import tqdm
from io import StringIO
from bs4 import BeautifulSoup

In [2]:
def get_new_def():
    list_files = []

    urlpage_mol = 'https://www.exomol.com/data/molecules/'
    r_mol = requests.get(urlpage_mol)
    if r_mol.status_code==200:
        page_mol = urllib.request.urlopen(urlpage_mol)
    soup_mol = BeautifulSoup(page_mol, 'html.parser')
    list_group_mol = soup_mol.find_all('div', attrs={'class': 'list-group'})
    list_molecule = []
    for i_mol in range(len(list(list_group_mol))):
        n_mol = len(list(list(list_group_mol)[i_mol]))
        list_group_item_mol = list(list(list_group_mol)[i_mol])[1:n_mol:2]
        str_group_mol = str(list_group_item_mol).split('"')
        list_molecule += str_group_mol[3:len(str_group_mol):4]

    for index_mol in tqdm(range(len(list_molecule))):    
        urlpage_iso = urlpage_mol + list_molecule[index_mol]
        print(list_molecule[index_mol])
        r_iso = requests.get(urlpage_iso)
        if r_iso.status_code==200:
            page_iso = urllib.request.urlopen(urlpage_iso)
        soup_iso = BeautifulSoup(page_iso, 'html.parser')
        list_group_iso = soup_iso.find_all('div', attrs={'class': 'list-group'})
        list_iso = []
        for i_iso in range(len(list(list_group_iso))):
            n_iso = len(list(list(list_group_iso)[i_iso]))
            list_group_item_iso = list(list(list_group_iso)[i_iso])[1:n_iso:2]
            str_group_iso = str(list_group_item_iso).split('"')
            list_iso += str_group_iso[3:len(str_group_iso):4]

        if (len(list_iso) != 0):
            for index_iso in range(len(list_iso)):
                urlpage_dataset = urlpage_iso + '/' + list_iso[index_iso]
                r_dataset = requests.get(urlpage_dataset)
                if r_dataset.status_code==200:
                    page_dataset = urllib.request.urlopen(urlpage_dataset)
                soup_dataset = BeautifulSoup(page_dataset, 'html.parser')
                list_group_dataset = soup_dataset.find_all('div', attrs={'class': 'list-group'})
                list_datasets = []
                for i_dataset in range(len(list(list_group_dataset))):
                    n_dataset = len(list(list(list_group_dataset)[i_dataset]))
                    list_group_item_dataset = list(list(list_group_dataset)[i_dataset])[1:n_dataset:2]
                    str_group_dataset = str(list_group_item_dataset).split('"')
                    list_datasets += str_group_dataset[3:len(str_group_dataset):4]
                list_dataset = list_datasets
                a = 'list-group-item link-list-group-item recommended'
                b = 'list-group-item link-list-group-item'
                c = 'pull-right'
                if a in list_dataset:
                    list_dataset.remove('list-group-item link-list-group-item recommended')
                if b in list_dataset:
                    list_dataset.remove('list-group-item link-list-group-item')
                if c in list_dataset:
                    list_dataset.remove('pull-right')

                if (len(list_dataset) != 0):
                    for index_dataset in range(len(list_dataset)):
                        urlpage_file = urlpage_dataset + '/' + list_dataset[index_dataset]
                        r_file = requests.get(urlpage_file)
                        if r_file.status_code==200:
                            page_file = urllib.request.urlopen(urlpage_file)
                        soup_file = BeautifulSoup(page_file, 'html.parser')
                        list_group_file = soup_file.find_all('div', attrs={'class': 'list-group'})

                        if (len(list_group_file) != 0):
                            for i_file in range(len(list(list_group_file))):
                                n_file = len(list(list(list_group_file)[i_file]))
                                list_group_item_file = list(list(list_group_file)[i_file])[1:n_file:2]
                                str_group_file = str(list_group_item_file).replace('/db','https://www.exomol.com/db').split('"')
                                list_files += str_group_file[3:len(str_group_file):4]

    list_file = list_files[0:len(list_files):2]
    new_def_url = pd.DataFrame()
    new_def_url['new'] = list_file
    return new_def_url

In [3]:
def get_old_def():
    exomol_all_url = 'http://www.exomol.com/db/exomol.all'
    content = requests.get(exomol_all_url).text.replace('#','')
    exomol_col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6']
    exomol_all = pd.read_csv(StringIO(content), sep='\\s+', names=exomol_col_name, header=None)
    
    first = exomol_all['c1']
    third = exomol_all['c3']
    row = len(first)
    iso_slug = pd.DataFrame()
    iso_formula = pd.DataFrame()
    isotopologue = pd.DataFrame()
    molecule_single = pd.DataFrame()
    num_isotopologues = pd.DataFrame()

    for i in range(row):

        _iso_slug = exomol_all[first.isin(['Iso-slug'])]['c0'].values
        _iso_formula = exomol_all[first.isin(['IsoFormula'])]['c0'].values
        _isotopologue = exomol_all[first.isin(['Isotopologue'])]['c0'].values
        _molecule_single = exomol_all[first.isin(['Molecule'])]['c0'].values
        _num_isotopologues = exomol_all[third.isin(['isotopologues'])]['c0'].values

    iso_slug = iso_slug.append(pd.DataFrame(_iso_slug))
    iso_formula = iso_formula.append(pd.DataFrame(_iso_formula))
    isotopologue = isotopologue.append(pd.DataFrame(_isotopologue))
    molecule_single = molecule_single.append(pd.DataFrame(_molecule_single))
    num_isotopologues = num_isotopologues.append(pd.DataFrame(_num_isotopologues))
    
    molecule_repeated = pd.DataFrame()
    molecule_num = len(molecule_single)

    for j in range(molecule_num):    
        molecule_repeated = molecule_repeated.append(pd.DataFrame((molecule_single.values[j] + ' ')
                                                                  * int(num_isotopologues.values[j])))

    molecule_str = (str(molecule_repeated.values).replace("[['"," ")
                    .replace("']\n ['"," ").replace("']]"," ").replace("+","_p"))
    molecule = pd.read_csv(StringIO(molecule_str), sep='\s+', header=None)
    

    old_def_url = pd.DataFrame()
    old_def_num = len(iso_slug)
    for i in range(old_def_num):
        old_def_url = old_def_url.append('https://www.exomol.com/db/' + molecule[i] + '/'
                                         + iso_slug.values[i] + '/'+ isotopologue.values[i] + '/'
                                         + iso_slug.values[i] + '__' + isotopologue.values[i] + '.def')
    old_def_url.columns = ['old']
    return old_def_url

In [4]:
def get_add_def():
    add_def_url = new_def_url[~(new_def_url['new'].isin(old_def_url['old']))].reset_index(drop=True)
    add_def_url.columns = ['add']
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', 1000)
    pd.set_option('display.max_colwidth', 1000)
    return add_def_url

In [5]:
def download_master_file():
    folder_name = '../data/'
    master_filename = 'exomol.all'
    if os.path.exists(folder_name):
        pass
    else:
        os.makedirs(folder_name, exist_ok=True)
    filename = os.path.join(folder_name, master_filename)

    exomol_all_url = 'http://www.exomol.com/db/exomol.all'
    r = requests.get(exomol_all_url) 
    with open(filename, 'wb') as code:
        code.write(r.content)    


In [6]:
def get_invalid_urlpage():
    # Invalid URL page
    urlpages_list = []
    for i in tqdm(range(len(list(add_def_url['add'])))):
        urlpage = add_def_url['add'][i]
        r_add = requests.get(urlpage)
        if r_add.status_code!=200:
            urlpages_list.append(urlpage)
    urlpage = pd.DataFrame(urlpages_list)
    print('There are ', len(urlpage),' invalid url pages of def files.')
    return urlpage

In [7]:
new_def_url = get_new_def()
old_def_url = get_old_def()
add_def_url = get_add_def()

  0%|                                                                                           | 0/85 [00:00<?, ?it/s]

MgH


  1%|▉                                                                                  | 1/85 [00:32<45:15, 32.33s/it]

NaH


  2%|█▉                                                                                 | 2/85 [00:50<38:46, 28.03s/it]

...

 28%|███████████████████████▏                                                          | 24/85 [10:44<40:04, 39.41s/it]

SiO


 29%|████████████████████████                                                          | 25/85 [11:36<43:06, 43.11s/it]

CaO


KeyboardInterrupt: 

In [ ]:
print('New version of master file ExoMol.all has ', int(len(new_def_url)), ' def files.')
new_def_url

In [ ]:
print('Old version of master file ExoMol.all has ', len(old_def_url), ' def files.')
old_def_url

In [ ]:
print('Master file ExoMol.all has ', len(add_def_url), ' new def files.')
add_def_url

In [ ]:
download_master_file()

In [ ]:
urlpage = get_invalid_urlpage()
urlpage